# Previsão de Custos Médicos com Regressão Linear

Este notebook tem como objetivo desenvolver um modelo de regressão capaz de prever o valor dos custos médicos individuais cobrados pelo seguro de saúde, com base em características como idade, gênero, IMC, número de filhos, hábito de fumar e região.

As etapas seguidas incluem:
- Exploração dos dados
- Pré-processamento
- Modelagem e avaliação
- Validação estatística
- Visualizações e relatório final

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

df = pd.read_csv("dados_custos_medicos_simulados.csv")
df.head()

In [ ]:
df.info()
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.histplot(df["encargos"], kde=True)
plt.title("Distribuição dos Encargos")
plt.show()

In [ ]:
df.columns = ['idade', 'genero', 'imc', 'filhos', 'fumante', 'regiao', 'encargos']

In [ ]:
df.isnull().sum()

In [ ]:
df = df.replace({
    'genero': {'feminino': 'female', 'masculino': 'male'},
    'fumante': {'sim': 'yes', 'não': 'no'},
    'regiao': {'sudeste': 'southeast', 'sudoeste': 'southwest',
               'nordeste': 'northeast', 'noroeste': 'northwest'}
})

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df.drop("encargos", axis=1)
y = df["encargos"]

num_atributos = ["idade", "imc", "filhos"]
cat_atributos = ["genero", "fumante", "regiao"]

preprocessador = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_atributos),
    ("cat", OneHotEncoder(), cat_atributos)
])

modelo = Pipeline(steps=[
    ("preprocessador", preprocessador),
    ("regressor", LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

In [ ]:
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel("Encargos reais")
plt.ylabel("Encargos previstos")
plt.title("Previsões vs. Reais")
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.show()

In [ ]:
import statsmodels.api as sm

X_ = pd.get_dummies(X, drop_first=True)
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y, test_size=0.2, random_state=42)

X_train_sm = sm.add_constant(X_train_)
ols_model = sm.OLS(y_train_, X_train_sm).fit()
print(ols_model.summary())

## Conclusão

Com base nos resultados:
- O modelo apresentou uma boa capacidade preditiva com base no R².
- As variáveis mais influentes nos custos são o hábito de fumar e o IMC.
- O modelo pode ser utilizado como base para estimativas de seguros médicos individuais.

É possível aprimorar o modelo com mais dados e com modelos mais robustos como Árvores de Regressão, Random Forest ou XGBoost.